In [2]:
import numpy as np
import math

vocabulary = ['a','b','c','d','e','f','g','h','i','j','k',
             'l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' ']
language = ['e','s','j']

In [3]:
def read_file(file_name, path = './languageID/'):
    f = open(path+file_name, 'r')
    content = f.read()
    content = content.replace('\n', '')
    return content

In [4]:
def get_boc(file):
    """
    get bag of character for input file
    """
    l = len(file)
    vocb = dict()
    for i in range(l):
        if file[i] in vocb:
            vocb[file[i]] += 1
        else:
            vocb[file[i]] = 1
    return vocb

In [5]:
def get_embedding(file):
    """
    use counts of characters as embedding
    emb: embedding array of size of 27(a-z and ' ')
    """
    vocb = get_boc(file)
    emb = []
    for v in vocabulary:
        if v not in vocb:
            emb.append(0)
        else:
            emb.append(vocb[v])
    return emb

In [6]:
print(get_boc(f))
print(get_embedding(f))

NameError: name 'f' is not defined

# Question 3.1

In [7]:
e_training = []
s_training = []
j_training = []
for i in range(10):
    e_training.append(read_file("e"+str(i)+'.txt'))
    s_training.append(read_file("s"+str(i)+'.txt'))
    j_training.append(read_file("j"+str(i)+'.txt'))

In [8]:
training = {'e':e_training,'s':s_training,'j':j_training}

In [9]:
p_prior = (10+1)/(30+3)
log_prior = math.log(p_prior)

print(p_prior)

0.3333333333333333


# Question 3.2 & 3.3

In [10]:
raw_theta = {'e':[0]*27,'s':[0]*27,'j':[0]*27}
for l in language:
    for f in training[l]:
        tmp_emb = get_embedding(f)
        raw_theta[l] = [raw_theta[l][i] + tmp_emb[i] for i in range(27)]

In [11]:
print(raw_theta['e'])

[910, 168, 325, 332, 1594, 286, 264, 714, 838, 21, 56, 438, 310, 876, 975, 253, 8, 814, 1001, 1212, 403, 140, 234, 17, 209, 9, 2712]


In [12]:
# using log here to avoid underflow
log_theta = {'e':[0]*27,'s':[0]*27,'j':[0]*27}
for l in language:
    total = sum(raw_theta[l])
    # using add-1 smoothing
    log_theta[l] = [math.log((raw_theta[l][i]+1))-math.log((total+len(vocabulary))) for i in range(27)]

In [13]:
for l in log_theta:
#     print(log_theta[l])
    print("theta_"+l+":", [math.e**log_theta[l][i] for i in range(27)])
    print(sum([math.e**log_theta[l][i] for i in range(27)]))

theta_e: [0.0601478938333553, 0.011158061534398517, 0.02152383467582199, 0.021986002905057433, 0.10530833223293272, 0.018948897398653106, 0.017496368678198853, 0.04720718341476294, 0.05539416347550504, 0.0014525287204542451, 0.003763369866631453, 0.028984550376336977, 0.020533474184603183, 0.05790307671992602, 0.0644394559619701, 0.016770104317971733, 0.0005942162947312824, 0.053809586689554975, 0.06615608081341603, 0.08008715172322722, 0.026673709230159777, 0.009309388617456752, 0.01551564769576125, 0.0011884325894625637, 0.013865046877063242, 0.0006602403274792024, 0.17912320084510747]
0.9999999999999994
theta_s: [0.1045042824573295, 0.008256824203586173, 0.037525417462567025, 0.03974366874114241, 0.11374699611806025, 0.008626532750015403, 0.007209316655370021, 0.00455973873929386, 0.04984903567687472, 0.0066547538357261725, 0.0003080904553576929, 0.05292994023045165, 0.025817980158974674, 0.05416230205188243, 0.07246287510012941, 0.024277527882186217, 0.007702261383942331, 0.0592766

# Question 3.4

In [14]:
test = read_file('e10.txt')

In [15]:
test_emb = get_embedding(test)

In [16]:
print(test_emb)

[164, 32, 53, 57, 311, 55, 51, 140, 140, 3, 6, 85, 64, 139, 182, 53, 3, 141, 186, 225, 65, 31, 47, 4, 38, 2, 498]


# Question 3.5

In [17]:
log_p = {'e':0,'s':0,'j':0}
for l in language:
    tmp_log_theta = [log_theta[l][i]*test_emb[i] for i in range(27)]
    log_p[l] = sum(tmp_log_theta)
    print(l+": ", log_p[l])

e:  -7841.786386770362
s:  -8452.383194656028
j:  -8759.518886307718


# Question 3.7

In [18]:
e_test = []
s_test = []
j_test = []
for i in range(10,20):
    e_test.append(read_file("e"+str(i)+'.txt'))
    s_test.append(read_file("s"+str(i)+'.txt'))
    j_test.append(read_file("j"+str(i)+'.txt'))

In [19]:
test = {'e':e_test,'s':s_test,'j':j_test}

In [20]:
def get_log_theta(training,dim):
    """
    training process of this naive bayes model
    
    training: training set of a single class y
    dim: feature dimension
    return log(theta_i) for i in [1,d]
    
    using log() to avoid underflow
    """
    raw_theta = [0]*dim
    log_theta = [0]*dim
    for f in training:
        tmp_emb = get_embedding(f)
        raw_theta = [raw_theta[i] + tmp_emb[i] for i in range(27)]        
    total = sum(raw_theta)
    # using add-1 smoothing
    log_theta = [math.log((raw_theta[i]+1))-math.log((total+len(vocabulary))) for i in range(27)]
    return log_theta

In [21]:
log_theta = dict()
for l in language:
    log_theta[l] = get_log_theta(training[l],27)
    print(l)
    print(log_theta[l])

e
[-2.8109488524138104, -4.495593034750695, -3.838594368307061, -3.8173492596933247, -2.2508627344548238, -3.9660095339141472, -4.045761923687547, -3.053209206979761, -2.8932810432065628, -6.534449296315453, -5.582440481839218, -3.540992336598597, -3.8856988374945347, -2.8489847573015856, -2.7420291632606766, -4.088157482655232, -7.428267172337549, -2.922303636432906, -2.7157384680289587, -2.5246398407297184, -3.624076871712618, -4.6767318592956, -4.16590623552961, -6.735119991777604, -4.2783842189563, -7.322906656679723, -1.7196814370148372]
s
[-2.2585272279640334, -4.796715244364971, -3.2827367766059847, -3.2253047275201094, -2.1737786292530847, -4.752912621706578, -4.932381109518126, -5.390489951111712, -2.9987561272573906, -5.012423817191662, -8.085117131881782, -2.938786122331627, -3.6566841243937445, -2.915770146630705, -2.6246809158573097, -3.718204135017948, -4.86624130701358, -2.8255405936501754, -2.7219487930141284, -3.334981175643504, -3.3901062418939008, -5.130206852848046,

In [22]:
def predict(test_emb, log_theta, log_prior_p):
    """
    test_emb: test file's embedding
    log_theta & prior: dict of classes Y
    """
    highest_log_p = float("-inf")
    prediction = None
    for y in log_theta.keys():
        tmp_log_theta = [log_theta[y][i]*test_emb[i] for i in range(27)]
        log_p[y] = sum(tmp_log_theta)+log_prior_p[y]
        if log_p[y] > highest_log_p:
            highest_log_p = log_p[y]
            prediction = y
    return prediction

In [23]:
log_prior_p = {'e':math.log(1/3),'s':math.log(1/3),'j':math.log(1/3)}
e_prediction = []
for f in test['e']:
    test_emb = get_embedding(f)
    print(test_emb)
    e_prediction.append(predict(test_emb, log_theta, log_prior_p))
s_prediction = []
for f in test['s']:
    test_emb = get_embedding(f)
    s_prediction.append(predict(test_emb, log_theta, log_prior_p))
j_prediction = []
for f in test['j']:
    test_emb = get_embedding(f)
    j_prediction.append(predict(test_emb, log_theta, log_prior_p))

[164, 32, 53, 57, 311, 55, 51, 140, 140, 3, 6, 85, 64, 139, 182, 53, 3, 141, 186, 225, 65, 31, 47, 4, 38, 2, 498]
[199, 47, 70, 86, 352, 78, 47, 143, 170, 1, 15, 124, 59, 191, 236, 38, 3, 147, 194, 272, 86, 35, 57, 2, 43, 2, 618]
[122, 23, 59, 36, 196, 35, 15, 89, 103, 0, 8, 48, 38, 106, 111, 25, 1, 87, 124, 194, 58, 15, 24, 3, 25, 0, 348]
[97, 24, 25, 32, 173, 38, 23, 85, 87, 0, 7, 45, 42, 90, 127, 22, 2, 79, 94, 179, 49, 8, 29, 2, 28, 1, 299]
[113, 24, 33, 32, 164, 38, 33, 87, 87, 2, 5, 57, 45, 90, 104, 23, 1, 72, 109, 140, 36, 9, 23, 1, 19, 1, 322]
[108, 19, 34, 42, 157, 31, 29, 67, 101, 0, 8, 42, 49, 91, 100, 27, 2, 84, 96, 149, 38, 16, 29, 0, 23, 1, 271]
[159, 36, 66, 60, 316, 45, 35, 138, 138, 0, 14, 97, 53, 164, 168, 53, 0, 146, 169, 219, 56, 27, 46, 7, 32, 0, 477]
[155, 26, 62, 53, 267, 58, 41, 110, 128, 3, 14, 65, 63, 149, 174, 45, 3, 107, 145, 200, 55, 22, 30, 2, 30, 1, 416]
[126, 16, 31, 37, 166, 32, 21, 57, 98, 0, 7, 57, 43, 100, 90, 29, 0, 84, 105, 118, 46, 15, 24, 3, 23, 

In [194]:
print(e_prediction)
print(s_prediction)
print(j_prediction)
print(len(e_prediction))

['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e']
['s', 's', 's', 's', 's', 's', 's', 's', 's', 's']
['j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j']
10


# Question 3.8

In [225]:
import random
random.seed(2019)
class_num = random.randint(0,3)
num = random.randint(10,19)
file_name = language[class_num]+str(num)+".txt"

In [226]:
test_set = read_file(file_name)

In [227]:
test_set

'la hinchazon o edema las ojeras las bolsas y las arrugas son los males que se instalan alrededor de los ojosla prudencia es imprescindible a la hora de afrontar la depilacion unas cejas excesivamente depiladas tardan mucho tiempo en recuperar su forma inicial crecen con menos densidad e incluso puede aparecer calvasesta zona tan fragil ademas de estar expuesta a las agresiones del sol el frio el viento la humedad  la contaminacion y el aire acondicionado tambien se ve afectada por la falta se sueno el estres el tabaco el alcohol y acumulacion de horas frente al ordenadorno existe mirada perfecta sin unas cejas cuidadas con mimo las cejas enmarcan la mirada son fundamentales en la expresion del rostro y  requieren un cuidado pequeno aunque constanteprovocadas por venas bajo la piel las ojeras resultan muy antiesteticas el te la manzanilla el aloe vera y la calendula son algunos remedios naturales aunque en casos graves se puede acudir a la cirugia estetica'

In [229]:
shuffle_test_set = list(test_set)
random.shuffle(shuffle_test_set)
shuffle_test_set = ''.join(shuffle_test_set)

In [230]:
shuffle_test_set

'aenml ecsmlt lote iao j ltf c naelurnunuaoir    em uc r ee rlaeebar nn d aleeedlo sm me  lglarsi l aiceopod rroons aolioss eeiounmlhtma oao oiraoa dr  ceas  ustroore t ednuasetal  d ecads acs d subicdl d tsmuliq  vicaoompaeo aeeo desaledpeeoemta i uihpslala tocloumca scsld oalnecner ad cdeacotxa upldu tetea eaaao eut connente ecaerssanrs  elamdeanse lolsofe  pcalunaaaaol u oni neonann jevarsife ayeecyldrarlcs liouo p y ernlilio s etsean   caesaosnchc narvsadvcu aabssnla i uneh naes   e  oertgqaltarpuirr rueeorleuh   iaaai  rsilniniaibedoe rld usdejarvsaxs o ajiaeddseal sn eoai lt ecmerfadonsn esnnesac izrcup aeennuv se aamdaszvoulaaapialadust aeese oaanclsaar oear   mesjdrsnsnpcipeaaor iei pjlsem  o taan asaadlc malaaqlmid eqye eopngatlaerfdnlcf  rlnetnaaertsldlmtn entfaso msnciirnxninh iirnes ehianueuuoncalssczua   east sas garoogi ecnero  afaee er qad eeieaoa   aei pesdpra asaaj  cvmbnls xuc oaterft rtl  sg iu dplrsyssdeldlnae e an  ondacyoamrini c  dte'

In [234]:
test_emb = get_embedding(test_set)
shuffle_emb = get_embedding(shuffle_test_set)
print(test_emb,shuffle_emb)

[122, 5, 39, 39, 107, 9, 6, 7, 45, 7, 0, 58, 22, 61, 58, 17, 5, 48, 67, 31, 33, 8, 0, 4, 6, 3, 162] [122, 5, 39, 39, 107, 9, 6, 7, 45, 7, 0, 58, 22, 61, 58, 17, 5, 48, 67, 31, 33, 8, 0, 4, 6, 3, 162]


In [232]:
pred = predict(test_emb, log_theta, log_prior_p)
shuffle_pred = predict(shuffle_emb,log_theta,log_prior_p)

In [233]:
print(pred,shuffle_pred)

s s


# Question 4

In [29]:
def txt2Wekatxt(file):
    """
    Assuem the input file only has ' ' and a-z
    """
    c_list = []
    for i in range(len(file)):
        if file[i] != ' ':
            c_list.append(file[i])
            c_list.append(' ')
        else:
            c_list.append('space')
            c_list.append(' ')
    return ''.join(c_list)

In [35]:
for l in language:
    for i in range(20):
        tmp = read_file(l+str(i)+'.txt')
        out = txt2Wekatxt(tmp)
        f = open("./Weka text/"+l+"/"+l+str(i)+'.txt', 'w')
        f.write(out)
        f.close()

In [28]:
print(test['e'][1])

 the best explanation that we have today of this continuous accretion of energy is that it is due to shrinkage of the suns bulk under the force of gravity gravity is one of the most mysterious forces of nature but it is an obvious fact that bodies behave as if they attracted one another and newton worked out the law of this attraction we may say without trying to go too deeply into things that every particle of matter attracts every other throughout the universe if the diameter of the sun were to shrink by one mile all round this would mean that all the millions of tons in thepg  outer onemile thickness would have a straight drop of one mile towards the centre and that is not all because obviously the layers below this outer mile would also drop inwards each to a less degree than the one above it what a tremendous movement of matter however slowly it might take place and what a tremendous energy would be involved astronomers calculate that the above shrinkage of one mile all round woul